In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import (CountVectorizer,
                                             TfidfVectorizer)
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from os import getcwd

In [3]:
RANDOM_STATE = 40

In [4]:
DATA_PATH = getcwd()
DATA_PATH = DATA_PATH.replace("notebooks", "")
DATA_PATH += "data"
SHEETS_PATH = DATA_PATH + "\\sheets"

In [5]:
data = pd.read_excel(f"{SHEETS_PATH}\\subset_data.xlsx")
data.fillna("#NA", inplace=True)

In [6]:
data.head()

,URL,final_statement_cleaned,final_statement_cleaned_2,0.99%dup
0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,holdinc europe ltd modern slavery act transpar...,holdinc europe ltd modern slavery act transpar...,False
1,https://1spatial.com/who-we-are/legal/modern-s...,independent research edison investments modern...,independent research edison investments modern...,False
2,https://www.shazans.com/slavery-and-human-traf...,slavery human trafficking statement slavery hu...,slavery human traffic statement slavery human ...,False
3,https://www.business-humanrights.org/sites/def...,modern slavery atement modern slavery atement ...,modern slavery atement modern slavery atement ...,False
4,https://www.2agriculture.com/wp-content/upload...,modern slavery act slavery human trafficking s...,modern slavery act slavery human traffic state...,False


In [7]:
N_GRAMS = (1, 1)
count_vect = CountVectorizer(ngram_range=N_GRAMS)
X = count_vect.fit_transform(data["final_statement_cleaned_2"].values) 

In [8]:
N_TOPICS = 10
LDA = LatentDirichletAllocation(n_components=N_TOPICS, n_jobs=-1, verbose=1, random_state=RANDOM_STATE)
LDA.fit(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_jobs=-1, random_state=40, verbose=1)

In [10]:
N_TOP_WORDS = 15 
feature_names = count_vect.get_feature_names()
topic_components = LDA.components_ 
# Note: topic words are words with high LDA component values
for i in range(N_TOPICS):
    print(f"Topic {i}: {[feature_names[j] for j in np.argsort(topic_components[i])[::-1][:N_TOP_WORDS]]}")
    print()

Topic 0: ['service', 'us', 'contact', 'support', 'care', 'news', 'new', 'solutions', 'home', 'people', 'career', 'products', 'energy', 'water', 'help']

Topic 1: ['slavery', 'modern', 'traffic', 'human', 'supply', 'chain', 'business', 'ensure', 'statement', 'policy', 'act', 'risk', 'suppliers', 'take', 'within']

Topic 2: ['use', 'term', 'may', 'site', 'information', 'shall', 'website', 'service', 'content', 'condition', 'include', 'right', 'order', 'time', 'without']

Topic 3: ['human', 'suppliers', 'supply', 'conduct', 'chain', 'traffic', 'code', 'supplier', 'labor', 'slavery', 'business', 'compliance', 'include', 'audit', 'employees']

Topic 4: ['www', 'https', 'com', 'uk', 'en', 'cking', 'class', 'store', 'div', 'co', 'tra', 'http', 'span', 'brand', 'shop']

Topic 5: ['information', 'use', 'data', 'us', 'personal', 'cookies', 'may', 'website', 'site', 'service', 'privacy', 'please', 'provide', 'contact', 'process']

Topic 6: ['service', 'firm', 'financial', 'include', 'risk', 'coun